In [1]:
import numpy
import torch
import mobster as mb

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
data = mb.load_example_data("./")


In [4]:
import pyro

pyro.enable_validation(True)
fitted = mb.fit_mobster(data,2,1, subclonal_prior="Moyal", multi_tail = False,
                        purity = 0.4,  truncated_pareto=True, max_it = 1000,
                        lr = 0.0001,  e = 0.00001, lrd_gamma = 0.01,
        number_of_trials_subclonal= 1200, prior_lims_k=[1.,10000.],
                        prior_lims_clonal=[1,10000.],
                        number_of_trials_clonal_mean= 600)


Running MOBSTER on 5 karyotypes with 2 subclones.
Fitting a model with tail


ELBO: 3108.296197055  :  59%|█████▉    | 588/1000 [00:24<00:16, 24.27it/s]

Computing cluster assignements.


Computing information criteria.
Done!



In [32]:


mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [24]:
fitted["model_parameters"]["2:1"]["mixture_probs"]


array([0.5, 0.5], dtype=float32)

In [6]:

pyro.get_param_store()

In [26]:
(0.64721787 * 0.4) / (2 * 0.6 + 0.4)

0.1618044675